In [150]:
import sklearn
import numpy as np

In [151]:
from sklearn.datasets import load_iris
datasets = load_iris()

In [152]:
x_data = datasets["data"]
x_data[:5]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2]])

In [153]:
y_data = datasets["target"]
y_data

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [154]:
y_data = y_data.reshape([-1,1])
y_data[:3]

array([[0],
       [0],
       [0]])

In [155]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(y_data)  
y_data = enc.transform(y_data).toarray()
y_data[:3]

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]])

In [156]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
x_data_minmax = min_max_scaler.fit_transform(x_data)
x_data_minmax[:3]

array([[ 0.22222222,  0.625     ,  0.06779661,  0.04166667],
       [ 0.16666667,  0.41666667,  0.06779661,  0.04166667],
       [ 0.11111111,  0.5       ,  0.05084746,  0.04166667]])

In [157]:
x_0 =np.ones(x_data_minmax.shape[0])
x_data_minmax = np.column_stack((x_0, x_data_minmax))

x_data_minmax[:3]

array([[ 1.        ,  0.22222222,  0.625     ,  0.06779661,  0.04166667],
       [ 1.        ,  0.16666667,  0.41666667,  0.06779661,  0.04166667],
       [ 1.        ,  0.11111111,  0.5       ,  0.05084746,  0.04166667]])

In [158]:
weights = np.random.uniform(size=(3,5))
weights

array([[ 0.42415538,  0.95769226,  0.38126014,  0.49730859,  0.11499596],
       [ 0.89003746,  0.63295762,  0.78002944,  0.48955023,  0.18905427],
       [ 0.89938368,  0.28215225,  0.96737798,  0.46710566,  0.07710664]])

In [159]:
def softmax(z):
    e = np.exp(z)
    p = z / np.sum(z, axis=1).reshape([-1,1])
    return p

In [160]:
z = x_data_minmax.dot(weights.T)
z[:3]

array([[ 0.91377081,  1.55928022,  1.60157638],
       [ 0.78113649,  1.36160977,  1.38436417],
       [ 0.75127408,  1.38315045,  1.44138683]])

In [161]:
softmax(z)[:3]

array([[ 0.22425874,  0.38268044,  0.39306082],
       [ 0.22146641,  0.38604115,  0.39249244],
       [ 0.21009891,  0.38680744,  0.40309364]])

In [162]:
def cross_entropy_function(y, x, weights):
    z = x_data_minmax.dot(weights.T)
    result = - np.sum(np.sum((y * np.log(softmax(z))), axis=1).reshape((-1,1)))
    return result 

In [163]:
cross_entropy_function(y_data,x_data_minmax,weights)

179.09685212325337

In [164]:
def minimize_grdient(y, x, initial_weights, iterations = 15000, alpha=0.001):
    cost_history= []
    theta_history = []
    m = y.shape[0]
    theta = np.copy(initial_weights)
    
    number_of_classes = theta.shape[0]
    number_of_weights = theta.shape[1]
    
    for _ in range(iterations):
        original_theta = np.copy(theta)
        for k in range(number_of_classes):        
            for j in range(number_of_weights):
                partial_x = x[:, j]
                partial_entropy = y - softmax(x.dot(original_theta.T))
                theta[k][j]  = original_theta[k][j] + (alpha* partial_entropy[:,k].dot(partial_x.T) ) /150
        if (_ % 1000) == 0:
            print(cross_entropy_function(y,x,theta)/150)
            cost_history.append(cross_entropy_function(y,x,theta))
    return theta, cost_history

            

In [165]:
# weights = minimize_grdient(y_data, x_data_minmax,weights)
theta, cost_history = minimize_grdient(y_data, x_data_minmax,weights)

1.19392462452
1.14369009852
1.10004068158
1.061278132
1.02637396278
0.994645589985
0.965605008972
0.93888288033
0.914187813784
0.891282999967
0.869971857993
0.85008863877
0.831491918695
0.814059882939
0.797686780842


In [166]:
rand_index= np.random.randint(0,150,30)
np.argmax(softmax(x_data_minmax[rand_index].dot(theta.T)),axis=1) 

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 2, 2])

In [167]:
np.argmax(y_data[rand_index],axis=1)

array([1, 0, 1, 2, 1, 2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 1, 1, 0, 2, 1, 2, 2, 2,
       1, 2, 2, 0, 2, 2, 2])